## Import packages and modules

In [7]:
# Import necessary packages and modules

# General packages
import numpy as np
import random
import sys
import os
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)
import numpy as np
import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots
from constants.constants import exports_dir

# Import custom modules
from modules.Account import Account
from modules.Dark import DarkToken
from modules.FuelCells import FuelCellsToken
from modules.JourneyPhaseManager import JourneyPhaseManager
from modules.Treasury import Treasury
from modules.LaunchControlCenter import LaunchControlCenter
from modules.EvilAddress import EvilAddress
from modules.Jackpot import Jackpot
from modules.Pool import Pool
from modules.Team import Team
from modules.DAIStableCoin import DAIStableCoin

# Set up initial configurations for the simulation
np.random.seed(42)
random.seed(42)


## Initialize accounts

In [8]:
# Import necessary packages and modules

# Initialize Accounts

# Public account
public_account = Account()

# MagicBox account managed by the team
magic_box_account = Account()

# Team account to collect share of Dark tokens used to mint NFTs
team_account = Team()

# Initialize Dark token
dark_token = DarkToken()

# Mint initial Dark token supply to Public, MagicBox, and Evil address
initial_dark_supply = 1000000  # Total initial supply
public_initial_balance = int(initial_dark_supply * 0.10)
magic_box_initial_balance = int(initial_dark_supply * 0.25)
evil_address_initial_balance = int(initial_dark_supply * 0.65)

dark_token.mint(public_account, public_initial_balance)
dark_token.mint(magic_box_account, magic_box_initial_balance)

# Initialize FuelCells token
fuel_cells_token = FuelCellsToken()

# Initialize JourneyPhaseManager
journey_phase_manager = JourneyPhaseManager()

# Initialize Treasury
treasury_account = Account()
treasury = Treasury(treasury_account, dark_token)

# Initialize Jackpot
jackpot_account = Account()
jackpot = Jackpot(jackpot_account, dark_token, journey_phase_manager, fuel_cells_token)

# Initialize LaunchControlCenter
lcc = LaunchControlCenter(dark_token, fuel_cells_token, treasury_account, jackpot_account, team_account, journey_phase_manager)

# Initialize EvilAddress
evil_address_account = EvilAddress(dark_token, fuel_cells_token, lcc, journey_phase_manager, evil_address_initial_balance)
dark_token.mint(evil_address_account, evil_address_initial_balance)

# Initialize Pool
dai_token = DAIStableCoin()
pool = Pool(dark_token, dai_token)

# Initialize accounts and variables summary
print("Initial Balances:")
print(f"Public Account: {dark_token.balance_of(public_account)} Dark tokens")
print(f"MagicBox Account: {dark_token.balance_of(magic_box_account)} Dark tokens")
print(f"Evil Address Account: {dark_token.balance_of(evil_address_account)} Dark tokens")
print(f"Team Account: {dark_token.balance_of(team_account)} Dark tokens")


Initial Balances:
Public Account: 100000 Dark tokens
MagicBox Account: 250000 Dark tokens
Evil Address Account: 650000 Dark tokens
Team Account: 0 Dark tokens


# Initialize Liquidity Pool

In [9]:
# Initialize Liquidity Pool

# Mint initial DAI supply to MagicBox account
initial_dai_supply = 300000  # Total initial DAI supply for liquidity
dai_token.mint(magic_box_account, initial_dai_supply)

# Define liquidity amounts
dark_liquidity_amount = 100000  # Dark tokens from MagicBox balance
dai_liquidity_amount = 300000   # DAI tokens

# Add liquidity to the pool from MagicBox
pool.add_liquidity(magic_box_account, dark_liquidity_amount, dai_liquidity_amount)

# Print pool status after adding liquidity
print(f"Liquidity Pool Initialized:")
print(pool)
print(pool.get_price())


Liquidity Pool Initialized:
Pool: Dark reserve = 100000, DAI reserve = 300000, Fee = 0.003, Total volume = 0
3.0


## Utility Functions

In [10]:
# Utility functions

# Helper function to simulate trades
def simulate_trades(account, pool, total_trades, buy_pressure, sell_pressure, nft_purchase_share_from_public_buy, journey_phase_manager, dai_token):
    for _ in range(total_trades):
        trade_type = np.random.choice(["buy", "sell"], p=[buy_pressure, sell_pressure])
        if trade_type == "buy":
            dai_amount = np.random.randint(100, 1000)  # Random DAI amount to buy Dark tokens
            if dai_token.balance_of(account) < dai_amount:
                dai_token.mint(account, dai_amount)  # Mint DAI tokens if account has insufficient balance
            if pool.dai_reserve >= dai_amount:
                pool.buy(account, dai_amount)
                dark_tokens_purchased = dai_amount / pool.get_price()
                nft_purchase_amount = int(dark_tokens_purchased * nft_purchase_share_from_public_buy)
                if journey_phase_manager.get_current_phase() == 1 and dark_tokens_purchased >= nft_purchase_amount:
                    lcc.mint_nft(account, nft_purchase_amount)
        else:
            dark_amount = np.random.randint(100, 1000)  # Random Dark amount to sell for DAI
            if dark_token.balance_of(account) >= dark_amount and pool.dark_reserve >= dark_amount:
                pool.sell(account, dark_amount)


## Variables

In [11]:
# Variables for controlling buy/sell pressures and trades

buy_pressure = 0.6  # 60% of the trades will be buys
sell_pressure = 0.4  # 40% of the trades will be sells
total_trades = 10000  # Total number of trades in the journey
nft_purchase_share_from_public_buy = 0.3  # 30% of the Dark tokens from buys will be used for minting NFTs
public_nft_mint_percentage = 0.10  # 10% of the public Dark token balance will be used to mint NFTs

# Initial pool status
initial_dark_reserve = pool.dark_reserve
initial_dai_reserve = pool.dai_reserve
initial_price = pool.get_price()

print(f"Initial Dark reserve: {initial_dark_reserve}")
print(f"Initial DAI reserve: {initial_dai_reserve}")
print(f"Initial Dark token price: {initial_price}")


Initial Dark reserve: 100000
Initial DAI reserve: 300000
Initial Dark token price: 3.0


## Run through the journeys

In [12]:
# Journey 1 Simulation

# Simulate the minting phase
public_initial_balance = dark_token.balance_of(public_account)
public_nft_mint_amount = int(public_initial_balance * public_nft_mint_percentage)  # 10% of the public Dark token balance

evil_address_nft_mint_amount = int(evil_address_account.starting_balance * evil_address_account.b * (evil_address_account.r ** (journey_phase_manager.get_current_journey() - 1)) / 100)

# Public mints NFTs
lcc.mint_nft(public_account, public_nft_mint_amount)

# Evil address mints NFTs
evil_address_account.mint_fuel_cells()

# Update phase to lottery phase
journey_phase_manager.increment_phase()

# Capture the jackpot balance before the lottery
initial_jackpot_balance = dark_token.balance_of(jackpot.account)

# Conduct the lottery
lottery_result = jackpot.conduct_lottery()
print(lottery_result)

# Total lottery payouts
total_lottery_payouts = sum(jackpot.calculate_payouts(journey_phase_manager.get_current_journey(), initial_jackpot_balance))
print(f"Total lottery payouts: {total_lottery_payouts}")

# Public sells all their lottery winnings
lottery_winnings = jackpot.get_lottery_winnings(journey_phase_manager.get_current_journey(), public_account)
if lottery_winnings > 0:
    pool.sell(public_account, lottery_winnings)

print(f"Total lottery winnings for public: {lottery_winnings}")

# Simulate trades in Phase 1
phase_1_trades = int(total_trades * 0.3)  # 30% of the trades in Phase 1
simulate_trades(public_account, pool, phase_1_trades, buy_pressure, sell_pressure, nft_purchase_share_from_public_buy, journey_phase_manager, dai_token)
print(f"Total volume in Pool at end of Phase 1: {pool.total_volume}")

# Update phase to yield distribution
journey_phase_manager.increment_phase()

# Simulate trades in Phase 2
phase_2_trades = int(total_trades * 0.5)  # 50% of the trades in Phase 2
simulate_trades(public_account, pool, phase_2_trades, buy_pressure, sell_pressure, nft_purchase_share_from_public_buy, journey_phase_manager, dai_token)
print(f"Total volume in Pool at end of Phase 2: {pool.total_volume}")

# Update phase to next journey
journey_phase_manager.increment_phase()

# Simulate trades in Phase 3
phase_3_trades = total_trades - (phase_1_trades + phase_2_trades)  # Remaining trades in Phase 3
simulate_trades(public_account, pool, phase_3_trades, buy_pressure, sell_pressure, nft_purchase_share_from_public_buy, journey_phase_manager, dai_token)
print(f"Total volume in Pool at end of Phase 3: {pool.total_volume}")

# Distribute yield to NFT holders
treasury.distribute_yield(journey_phase_manager)

# Calculate the total yield given by the treasury
total_yield_given = treasury.total_yield_distributed
print(f"Total yield given by the treasury in Journey 1: {total_yield_given}")

# Calculate the yield per NFT for public and evil address
public_nft_count = fuel_cells_token.balance_of(public_account)
evil_nft_count = fuel_cells_token.balance_of(evil_address_account)
total_nft_count = public_nft_count + evil_nft_count

yield_per_nft = total_yield_given / total_nft_count if total_nft_count else 0
print(f"Total yield per NFT: {yield_per_nft}")

# Final pool status
final_dark_reserve = pool.dark_reserve
final_dai_reserve = pool.dai_reserve
final_price = pool.get_price()

print(f"Final Dark reserve: {final_dark_reserve}")
print(f"Final DAI reserve: {final_dai_reserve}")
print(f"Final Dark token price: {final_price}")

# Print summary of Journey 1
print("Journey 1 Summary:")
print(f"Public Account: {fuel_cells_token.balance_of(public_account)} FuelCells NFTs")
print(f"Evil Address Account: {fuel_cells_token.balance_of(evil_address_account)} FuelCells NFTs")
print(f"Treasury Balance: {dark_token.balance_of(treasury_account)} Dark tokens")
print(f"Jackpot Balance: {dark_token.balance_of(jackpot_account)} Dark tokens")
print(f"Pool: {pool}")
print(f"Total trades: {total_trades}")
print(f"Buy pressure: {buy_pressure * 100}%")
print(f"Sell pressure: {sell_pressure * 100}%")
print(f"Total lottery winnings per winner: {lottery_winnings / total_trades if total_trades else 0}")
print(f"Total volume in Pool: {pool.total_volume}")

# Move to next journey
journey_phase_manager.increment_phase()


Lottery conducted and payouts distributed.
Total lottery payouts: 15407.330367000002
Total lottery winnings for public: 1997.4703783610328
Total volume in Pool at end of Phase 1: 2112386.2639784855
Total volume in Pool at end of Phase 2: 5521513.616443323
Total volume in Pool at end of Phase 3: 6724153.197771029
Total yield given by the treasury in Journey 1: 7789.553309999996
Total yield per NFT: 0.04388999999999998
Final Dark reserve: 91002.37016187528
Final DAI reserve: 319520.80222896655
Final Dark token price: 3.5111261570506573
Journey 1 Summary:
Public Account: 110441 FuelCells NFTs
Evil Address Account: 67038 FuelCells NFTs
Treasury Balance: 136658.8299999999 Dark tokens
Jackpot Balance: 20088.469633000055 Dark tokens
Pool: Pool: Dark reserve = 91002.37016187528, DAI reserve = 319520.80222896655, Fee = 0.003, Total volume = 6724153.197771029
Total trades: 10000
Buy pressure: 60.0%
Sell pressure: 40.0%
Total lottery winnings per winner: 0.19974703783610329
Total volume in Pool: 